# Python Intro - Final Exercise


In [416]:
import pyodbc
import pandas as pd
import numpy as np

In [417]:
### In windows, Using a ODBC DNS (predefined connection name)
conn =  pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=DESKTOP-VA3UHPO\SQLEXPRESS;"
                      "Database=COlleage;"
                      "Trusted_Connection=yes;")



In [418]:
### Get the students table
students = pd.read_sql("SELECT * FROM dbo.students;",conn)
students.head()
courses = pd.read_sql("SELECT * FROM dbo.courses;",conn)
courses.rename(columns={'DepartmentID' : 'DepartmentId'}, inplace=True)
classrooms = pd.read_sql("SELECT * FROM dbo.classrooms;",conn)
departments = pd.read_sql("SELECT * FROM dbo.departments;",conn)
teachers = pd.read_sql("SELECT * FROM dbo.teachers;",conn)

print(classrooms)

     CourseId  StudentId     degree
0           1        202  92.031070
1           1         12  81.175916
2           1        232  82.396587
3           1        123  62.420449
4           1        274  72.569900
..        ...        ...        ...
749        31         77  78.480534
750        31        108  79.660333
751        31        193  80.876061
752        31        233  92.495579
753        31         16  78.507968

[754 rows x 3 columns]


# Questions
 
 
## Q1. Count the number of students on each department

In [419]:
## Get the students by Department
courses.rename(columns={'DepartmentID' : 'DepartmentId'}, inplace=True)
df1 = pd.merge(classrooms,courses,on='CourseId')
df1 = pd.merge(df1,departments,on='DepartmentId')
gb_df1=df1.groupby('DepartmentName').agg({'StudentId':'nunique'})
gb_df1

,StudentId
DepartmentName,
Arts,137
English,84
Science,200
Sport,81


## Q2. How many students have each course of the English department and the total number of students in the department?

In [420]:
# Get the English courses per student 

df2 = pd.merge(classrooms,courses,on='CourseId')
df2 = df2[df2.DepartmentId == 1]

gb_df2 = df2.groupby('CourseId').agg({'StudentId':'count'})
print(gb_df2)
tot=len(df2.drop_duplicates('StudentId'))
print("-------------------")
print("total           ",tot)


          StudentId
CourseId           
1                32
2                24
3                38
-------------------
total            84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [421]:
df3= pd.merge(classrooms,courses,on='CourseId')
df3= df3[df3.DepartmentId == 2]

gb_df3 = df3.groupby('CourseName').agg({'StudentId' : 'count'})
gb_df3['clasroom_size']='Small classrooms'
gb_df3.loc[(gb_df3['StudentId']>=22),'clasroom_size']='Big_Clasrooms'
gb_df4=gb_df3.groupby('clasroom_size').agg('count')
gb_df4


,StudentId
clasroom_size,
Big_Clasrooms,10
Small classrooms,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [422]:
df4=students.groupby('Gender').agg({'StudentId':'count'})
df4

,StudentId
Gender,
F,165
M,115


## Q5. For which courses the percentage of male/female students is over 70%?

In [423]:
# Merge df:
df = pd.merge(classrooms,courses,on='CourseId', how='left')
df = pd.merge(df,departments,on='DepartmentId', how='left')
df = pd.merge(df,students,on='StudentId', how='left')
print(df.shape[0])

courses_count_df = df.groupby('CourseName')['StudentId'].count().reset_index()
courses_count_df = courses_count_df.rename(columns={'StudentId': 'course_count'})

course_gb = df.groupby(by=['CourseName','Gender'])['StudentId'].count().reset_index()
course_gb = pd.merge(course_gb,courses_count_df,on='CourseName', how='left')
course_gb['gender_prec'] = course_gb.apply(lambda x: (x['StudentId']/x['course_count']),axis=1)
over_gender_prec = course_gb[course_gb['gender_prec']>0.7].reset_index(drop=True)
over_gender_prec

754


,CourseName,Gender,StudentId,course_count,gender_prec
0,Sculpture,F,17,24,0.708333
1,Tenis,F,13,18,0.722222


## Q6. For each department, how many students passed with a grades over 80?

In [424]:

df = df.dropna()

df1 = df.sort_values(by='degree', ascending=False).reset_index(drop=True)
df1 =df.drop_duplicates(subset=['CourseId','StudentId'], keep='first')

df1['above_80'] = df1['degree'].apply(lambda x: 1 if x>80 else 0)

gb_df = df1.groupby(by='DepartmentName').agg({'StudentId':'nunique','above_80':'sum'}).reset_index()

gb_df['prec(%)'] = gb_df.apply(lambda x: round((x['above_80']/x['StudentId'])*100,1),axis=1)
gb_df



<ipython-input-424-8390c0e55f5a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['above_80'] = df1['degree'].apply(lambda x: 1 if x>80 else 0)


,DepartmentName,StudentId,above_80,prec(%)
0,Arts,137,66,48.2
1,English,84,43,51.2
2,Science,200,167,83.5
3,Sport,81,40,49.4


## Q7. For each department, how many students passed with a grades under 60?

In [425]:
df2 = df1.copy()
df2['student60']=0
df2.loc[(df2['degree']<60),'student60']=1
gb_df2=df2.groupby(['DepartmentName']).agg({'StudentId':'nunique','student60':'sum'})
gb_df2['prec(%)'] = gb_df2.apply(lambda x: round((x['student60']/x['StudentId'])*100,1),axis=1)
gb_df2

,StudentId,student60,prec(%)
DepartmentName,,,
Arts,137,3,2.2
English,84,2,2.4
Science,200,7,3.5
Sport,81,1,1.2


## Q8. Rate the teachers by their average student's grades (in descending order).

In [426]:

df8 = pd.merge(classrooms,courses,on='CourseId')
df8 = pd.merge(df8,teachers,on='TeacherId')
df8
gd_df8 = df8.groupby(['FirstName','LastName']).agg({"degree":"mean"})
gb_df8 = gd_df8.sort_values(['degree'] ,ascending = False)
gb_df8

,,degree
FirstName,LastName,
Jordyn,Baron,80.729494
Heather,Metcalf,80.440242
Kaylea,Monzingo,79.365537
Conor,Mckittrick,78.699492
Tanner,Fitzsimons,78.443395
Mackenzie,Olsen,78.383348
Madison,Batan,77.743575
Katerina,Young,77.406852
Mikayla,Russell,77.134323


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [427]:
df9 = pd.merge(departments,courses,on='DepartmentId')
df9 = pd.merge(df9,teachers,on='TeacherId')
df9 = pd.merge(df9,classrooms,on='CourseId')

gb_df9 = df9.groupby(['CourseName','DepartmentName','FirstName','LastName']).agg({'StudentId':'nunique'})
gb_df9

,,,,StudentId
CourseName,DepartmentName,FirstName,LastName,
Advanced English,English,Conor,Mckittrick,24
Algebra,Science,Alexander,Watts,12
Biology,Science,Jordyn,Baron,36
Chemistry,Science,Jeffrey,Batzel,23
Computer Science,Science,Kaylea,Monzingo,38
Data Science,Science,Mackenzie,Olsen,30
English Begginers,English,Conor,Mckittrick,32
Geometry,Science,Alexander,Watts,13
History of Arts,Arts,Caroline,Knuesel,32


## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [428]:
df10 = pd.merge(students,classrooms,on='StudentId', how='left')
df10 = pd.merge(df10,courses,on='CourseId', how='left')
df10.loc[(df10['DepartmentId']==1),'English_degree']=df10['degree']
df10.loc[(df10['DepartmentId']==2),'Science_degree']=df10['degree']
df10.loc[(df10['DepartmentId']==3),'Arts_degree']=df10['degree']
df10.loc[(df10['DepartmentId']==4),'Sports_degree']=df10['degree']

gb_df10 = df10.groupby(['StudentId','FirstName','LastName']).agg(
{'CourseId' : 'nunique', 'degree' : 'mean' ,'English_degree' : 'mean',
'Science_degree' : 'mean','Arts_degree' : 'mean', 'Sports_degree' : 'mean' })
gb_df10

,,,CourseId,degree,English_degree,Science_degree,Arts_degree,Sports_degree
StudentId,FirstName,LastName,,,,,,
1,Alexandria,Heller,4,81.337515,NaN,79.557508,86.677535,NaN
2,Cassie,Liska,5,79.137364,NaN,78.867115,79.542738,NaN
3,Hayley,Frank,3,78.793700,83.972245,76.204427,NaN,NaN
4,Maxwell,Kotch,3,71.481003,70.929801,74.157066,69.356144,NaN
5,Dylan,Boehm,3,76.718650,NaN,100.000000,64.494845,65.661106
...,...,...,...,...,...,...,...,...
276,Dylan,Bannister,3,76.784735,NaN,76.784735,NaN,NaN
277,Bryan,Medina,6,80.482335,NaN,81.298837,78.849330,NaN
278,Sydney,Blas,3,75.359189,NaN,72.578017,NaN,76.749775
